In [ ]:
import MDAnalysis
import numpy as np
import nglview as nv
import matplotlib.pyplot as plt
print("Using MDAnalysis version", MDAnalysis.__version__)
print("Using NGLView version", nv.__version__)

In [ ]:
u = MDAnalysis.Universe('md.tpr','md.xtc')

In [ ]:
v = nv.show_mdanalysis(u)
system = v.component_0
system.clear()
v.camera='orthographic'
v.center()
v.add_unitcell()

#SOL = u.select_atoms('resname 2PL or resname PRO2')
#WALL = u.select_atoms('resname WALL')
system.add_licorice(selection = u.atoms.indices)

In [ ]:
v.display()

In [ ]:
valdip = []; valdip2 = []; cgeomzp = []; cgeomzp2=[]; cchg = [];
frames = len(u.trajectory)
equil = 0 # allow 1000 frames = 100 ps for equilibration say

SOL = u.select_atoms('resname PRO2')
nm = SOL.residues[0].atoms.names
charges = SOL.residues[0].atoms.charges
N = len(charges) #number of atoms in the solvent molecule

for step in u.trajectory[equil:frames-1]:
    
    lines = SOL.atoms.positions;
    
    data = {}
    datum = 0

    for variable in nm:
        data[variable] = lines[datum::N] #get the atomic positions
        datum+=1
    
    locals().update(data)
    
    # dipole calculation
    CGEOM = sum(data.values())/N
    
    CGEOMdipVector=np.zeros((int(len(SOL)/N),3)) 
    for i in range(N):
        CGEOMdipVector+=((data[nm[i]]-CGEOM)/10)*charges[i]
    
    j = 0
    while j < len(lines) / N:
        valdip.append(CGEOMdipVector[j][2])
        cgeomzp.append(CGEOM[j][2])
        j += 1

In [ ]:
from scipy.stats import binned_statistic # This function allows the computation of the sum, mean, median, or other statistic of the values (or set of values) within each bin.
bin_means, bin_edges, binnumber = binned_statistic(cgeomzp, valdip, statistic='mean', bins=np.linspace(5,110,5000))

In [ ]:
bin_width = (bin_edges[1] - bin_edges[0]); bin_centers = bin_edges[1:] - bin_width/2; lxn=bin_centers/10
dipoles= bin_means
fig,ax = plt.subplots()
ax.plot(lxn,dipoles,lw=0.5)  

In [ ]:
rhocounts, rhobins, rhobars = plt.hist(cgeomzp, bins=bin_edges,histtype='step')

In [ ]:
bin_means3, bin_edges3, binnumber3 = binned_statistic(cgeomzp, valdip, statistic='count', bins=bin_edges) # changed from bins=seg
binvolume = bin_width/10 * u.trajectory[0].dimensions[0] * u.trajectory[0].dimensions[1] / 100
Pz = ((dipoles/binvolume)*(bin_means3/frames))

In [ ]:
plt.plot(lxn,Pz,lw=0.8)
plt.show()

In [ ]:
pmidav=np.mean((Pz[2300:2700]))
potential=Pz-pmidav

In [ ]:
eo = 8.8541878128e-12 #F/m = C/Vm
eo = 8.8541878128e-21 #C/Vnm
1/1.602176634e-19 # C in terms of elementary charges
eoinv = 1/(eo*1/1.602176634e-19)

import scipy.integrate as sci
potentialright=potential[len(potential)//2:]
lxnrev=lxn[::-1]; potentialrev=potential[::-1]
POTRIGHT=(sci.cumtrapz(potentialright,lxn[len(potential)//2:],initial=0)*eoinv) #18.1 is permittivity of free space *nm /C
POTLEFT=(sci.cumtrapz(potentialrev[len(potential)//2:],lxnrev[len(potential)//2:],initial=0)*eoinv)
plt.plot(lxnrev[len(potential)//2:],POTLEFT,lw=0.8)
plt.plot(lxn[len(potential)//2:],POTRIGHT,lw=0.8)

plt.axhline(0, color='black',linestyle='--')
plt.axhline(0, color='black',linestyle='--')
plt.show()

In [ ]:
min(POTLEFT[2150::])